In [24]:
import collections
import tensorflow as tf
import numpy as np
import json

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, GRU, LSTM, Bidirectional, Dropout, Activation, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [26]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4402936414572804604
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8856368297638390680
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8324380408932114808
physical_device_desc: "device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5"
xla_global_id: 2144165316
]


In [27]:
def load_data(path):
    input_file = path
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

english_sentences = load_data('/kaggle/input/language/data/english')
french_sentences = load_data('/kaggle/input/language/data/french')

In [28]:
english_sentences[:5]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .']

In [29]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')

print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [30]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [31]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [32]:
def preprocess(x,y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [33]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [34]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    #Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])

    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

#Train the neural network
simple_rnn_model = simple_model(
tmp_x.shape,
max_french_sequence_length,
english_vocab_size,
french_vocab_size)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step - accuracy: 0.4610 - loss: 2.6416 - val_accuracy: 0.6281 - val_loss: nan
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.6257 - loss: 1.3159 - val_accuracy: 0.6815 - val_loss: nan
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 87ms/step - accuracy: 0.6631 - loss: 1.1290 - val_accuracy: 0.6883 - val_loss: nan
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.6785 - loss: 1.0333 - val_accuracy: 0.6996 - val_loss: nan
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.6867 - loss: 0.9771 - val_accuracy: 0.7075 - val_loss: nan
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.7030 - loss: 0.9146 - val_accuracy: 0.7289 - val_loss: nan
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.7070 - loss: 0.8939 - val_accuracy: 0.7179 - val_loss: nan
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.6991 - loss: 0.8868 - val_accuracy: 0.7301 - val_l

In [35]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])


Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
new jersey est parfois calme en mois de il et est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [36]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    #Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])

    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(bd_rnn_model.summary())
batch_size=32

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=batch_size, epochs=10, validation_split=0.2)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 21, 256)        │       100,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 21, 1024)       │       263,168 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 21, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 21, 344)        │       352,600 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 716,376 (2.73 MB)

 Trainable params: 716,376 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.6296 - loss: 1.3440 - val_accuracy: 0.6908 - val_loss: nan
Epoch 2/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.6861 - loss: 0.9588 - val_accuracy: 0.7298 - val_loss: nan
Epoch 3/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.7013 - loss: 0.8834 - val_accuracy: 0.7222 - val_loss: nan
Epoch 4/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.7109 - loss: 0.8414 - val_accuracy: 0.7578 - val_loss: nan
Epoch 5/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.7283 - loss: 0.7943 - val_accuracy: 0.7547 - val_loss: nan
Epoch 6/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.7385 - loss: 0.7678 - val_accuracy: 0.7669 - val_loss: nan
Epoch 7/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 37s 11ms/step - accuracy: 0.7455 - loss: 0.7537 - val_accuracy: 0.8042 - val_loss: nan
Epoch 8/10
3447/3447 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - accuracy: 0.7594 - loss: 0.7111 - 

In [37]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
new jersey est parfois chaud en mois et il est il en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [38]:
def bidirectional_embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size + 1, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])

    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Build the model
embed_rnn_model = bidirectional_embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(embed_rnn_model.summary())

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 21, 256)        │        51,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 21, 512)        │       789,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 21, 1024)       │       525,312 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 21, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 21, 344)        │       352,600 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,718,616 (6.56 MB)

 Trainable params: 1,718,616 (6.56 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 138ms/step - accuracy: 0.6026 - loss: 2.0265 - val_accuracy: 0.8840 - val_loss: nan
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 133ms/step - accuracy: 0.8915 - loss: 0.3411 - val_accuracy: 0.9434 - val_loss: nan
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - accuracy: 0.9431 - loss: 0.1829 - val_accuracy: 0.9618 - val_loss: nan
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9609 - loss: 0.1264 - val_accuracy: 0.9689 - val_loss: nan
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9690 - loss: 0.1007 - val_accuracy: 0.9751 - val_loss: nan
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9734 - loss: 0.0852 - val_accuracy: 0.9777 - val_loss: nan
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9772 - loss: 0.0728 - val_accuracy: 0.9794 - val_loss: nan
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9791 - loss: 0.0667 - val_accu

In [39]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [42]:
import tensorflow as tf
tf.keras.models.save_model(embed_rnn_model, 'english_to_french_model.keras')
# Serialize English Tokenizer to JSON
with open('english_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(english_tokenizer.to_json(), ensure_ascii=False))
    
# Serialize French Tokenizer to JSON
with open('french_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(french_tokenizer.to_json(), ensure_ascii=False))
    
# Save max lengths
max_french_sequence_length_json = max_french_sequence_length
with open('sequence_length.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(max_french_sequence_length_json, ensure_ascii=False))

In [43]:
import keras
# Print version information used in saving code (if available)
print("Saving Keras version:", keras.__version__)
print("Saving TensorFlow version:", tf.__version__)


Saving Keras version: 3.3.3
Saving TensorFlow version: 2.16.1
